#Importing

In [12]:
import matplotlib.pyplot as plt
import fastai
import cv2
from fastai.vision import *
%matplotlib inline
import torchvision
%matplotlib inline

#Loading model

In [13]:
learn = load_learner("/Users/sandeep/Desktop/dataandmodles/models/ballDetection")

#Getting prediction

In [14]:
def unscaleCoord(imagePoint):
  '''
  Helper function for getPrediction
  Returns 
  -----------
  unscaleCoord : [Height,Width]
  '''
  s = torch.tensor([imagePoint.size[0]/2 , imagePoint.size[1]/2],dtype=torch.float32)
  scaledCoord = imagePoint.data
  unscaledCoord = (scaledCoord+1.) * (s)
  return unscaledCoord

In [15]:
def getScaleFactors(imgW, imgH, originalImgH , originalImgW):
    '''
    Helper function for getPrediction
    
    Return
    --------
    A tensor with given scalefactor (wdith,height). 
    '''
    return tensor(originalImgW/imgW , originalImgH/imgH);

In [16]:
def resizeFrame(frame):
    '''
    Helper function for getPrediction: that resizes the frames.
    Returns image of size (490, 360)
    '''
    resizedImg =  cv2.resize(frame[:] , (490 , 360))
    return resizedImg

In [17]:
def mapToOrginialImage(coord,scaleFactors):
    '''
      Helper function for getPrediction
      Returns coord (width, height)
    '''
    coordInOriginalImage = coord.flip(1) * scaleFactors
    return tuple(coordInOriginalImage[0].numpy())

In [18]:
def getPrediction(frame):
    '''
        Given an image, returns the ball coord (x,y)
    '''
    resizedImg  = resizeFrame(frame[:])
#     resizedImg  = frame
    resizedImg = cv2.cvtColor(resizedImg, cv2.COLOR_BGR2RGB)
    resizedImg = Image(pil2tensor(resizedImg, dtype=np.float32).div_(255))
    imagePoint = learn.predict(resizedImg)[0]
    scaleFactors = getScaleFactors(490,360,frame.shape[0],frame.shape[1])  
    unscaledCoord = unscaleCoord(imagePoint)
    coordInOriginalImage= mapToOrginialImage(unscaledCoord, scaleFactors);
    return coordInOriginalImage


#Puting prediction into original image

In [19]:
def drawPrediction(frame,coord):
    '''
    Given a frame and coordinate
    It draws a circle for you.
    '''
   # scaleFactors = getScaleFactors(490,360,frame.shape[0],frame.shape[1])   
    #coordInOriginalImage= mapToOrginialImage(coord, scaleFactors);
    frame = cv2.circle(frame,coord,30,(255,255,0),1);
    return frame

#Test

In [20]:
import import_ipynb
import TestBallLocalization
import DrawPose
import time 

In [21]:
TESTARGS = {
            'debugger' : False,
            'testImage1' : '/Users/sandeep/Desktop/dataandmodles/data/cavs.png',
            'testResultPath' : '/Users/sandeep/Desktop/dataandmodles/data/ballTest/Regression'
           }

In [22]:
def write_image(img,fileName):
    cv2.imwrite(f'{TESTARGS["testResultPath"]}/{fileName}.png' , img)

In [27]:
def runTest():
    totalTime = []
    yhat = []
    images, labels = TestBallLocalization.imgs , TestBallLocalization.labels
    frame_rsme_collection = []
    for index,frame in enumerate(images):
        st = time.time()
        ball_pos = getPrediction(frame)
        et = time.time()
        timeTaken = et - st
        
        ball_pos = list(ball_pos)
        print(f'{ball_pos},{labels[index]}')
        yhat.append(ball_pos)
        totalTime.append(timeTaken)
        #Drawing and saving image
        frame_rsme = TestBallLocalization.RMSE([ labels[index] ],[ball_pos])
        ball_pos = (int(ball_pos[0]), int(ball_pos[1]))
        DrawPose.drawBall(frame,frame.shape[1],frame.shape[0],tuple(ball_pos),(255,0,0),20)
        DrawPose.drawBall(frame,frame.shape[1],frame.shape[0],tuple(labels[index]),(0,0,255),20)
        write_image(frame,f'{round(frame_rsme,3)}')
        
        #Prep for min RMSE and MAX
        frame_rsme_collection.append(frame_rsme)
        
    
    #Final total RMSE and time  
    rmse = TestBallLocalization.RMSE(labels,yhat)
    totalTime = np.mean(np.array(totalTime)) 
    #Final Min and Max 
    frame_rsme_collection = np.array(frame_rsme_collection)
    minRmse = np.min(frame_rsme_collection)
    maxRmse = np.max(frame_rsme_collection)
    #Writing Results 
    content = f'RMSE:{rmse}\nMax RMSE:{maxRmse}\n{minRmse}\nMean Time:{totalTime}'
    TestBallLocalization.writeResults('RegressionTestResults',content)
    

In [28]:
#runTest()

[460.5025, 194.87712],[711, 207]
[457.8434, 207.6247],[1008, 341]
[428.83405, 205.70915],[853, 214]
[474.81528, 204.77089],[570, 146]
[466.25067, 202.26671],[676, 335]
[461.17722, 192.41132],[380, 160]
[456.52402, 200.44307],[538, 228]
[482.38788, 193.0059],[723, 219]
[455.2711, 197.43803],[516, 210]
[488.50742, 187.23819],[244, 179]
[472.5222, 202.20839],[612, 199]
[452.00732, 197.63353],[737, 199]
[483.83035, 190.24535],[621, 236]
[466.19818, 200.50049],[736, 407]
[456.43753, 198.79787],[249, 359]
[450.5728, 193.14905],[864, 206]
[432.68585, 206.80717],[830, 199]
[477.64758, 197.50977],[744, 177]
[474.39517, 196.86534],[727, 204]
[445.7015, 210.00102],[404, 163]
[463.96503, 192.16017],[619, 261]
[447.1644, 205.07297],[167, 169]
[479.3091, 190.46617],[285, 327]
[486.37296, 191.49149],[628, 196]
[452.19745, 194.72954],[783, 184]
[422.64548, 208.68431],[869, 179]
[486.83932, 224.23842],[436, 183]
[446.49634, 207.91785],[1030, 100]
[471.71048, 202.4637],[557, 382]
[477.75024, 189.2846],[

[470.3826, 191.46161],[767, 201]
[515.44934, 188.13542],[630, 88]
[429.72882, 210.02364],[331, 222]
[487.3782, 194.83359],[718, 183]
[484.43585, 191.02701],[787, 432]
[464.02753, 200.26973],[1079, 307]
[458.45886, 200.29007],[536, 113]
[453.72427, 200.5173],[737, 221]
[480.5616, 190.88551],[692, 112]
[467.84454, 199.99199],[586, 264]
[459.03235, 190.22728],[631, 213]
[425.566, 199.79999],[289, 252]
[509.05762, 186.52995],[712, 524]
[480.75238, 195.99101],[710, 186]
[472.54056, 199.2837],[951, 141]
[441.56387, 211.18765],[660, 209]
[480.2293, 190.55824],[467, 197]
[510.3601, 187.72948],[560, 177]
[438.04883, 210.93579],[763, 183]
[479.72833, 196.92316],[851, 308]
[503.97372, 188.31824],[631, 399]
